In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Tokens").getOrCreate()

In [4]:
from pyspark.ml.feature import Tokenizer

In [5]:
dataframe = spark.createDataFrame([
                                   (0, 'Spark is great'),
                                   (1, 'We are learning Spark'),
                                   (2, 'Spark is better than hadoop no doubt'),
], ['id','sentence'])

dataframe.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|      Spark is great|
|  1|We are learning S...|
|  2|Spark is better t...|
+---+--------------------+



In [8]:
# Tokenize sentences
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
tokenizer

Tokenizer_d828d31f195f

In [37]:
# Transform and show dataframe
tokenized_df = tokenizer.transform(dataframe)
tokenized_df = remover.transform(tokenized_df)
tokenized_df.show(truncate=False)

+---+------------------------------------+--------------------------------------------+------------------------------+
|id |sentence                            |words                                       |filtered                      |
+---+------------------------------------+--------------------------------------------+------------------------------+
|0  |Spark is great                      |[spark, is, great]                          |[spark, great]                |
|1  |We are learning Spark               |[we, are, learning, spark]                  |[learning, spark]             |
|2  |Spark is better than hadoop no doubt|[spark, is, better, than, hadoop, no, doubt]|[spark, better, hadoop, doubt]|
+---+------------------------------------+--------------------------------------------+------------------------------+



In [36]:
remover.transform(tokenized_df).show(truncate=False)

+---+------------------------------------+--------------------------------------------+------------------------------+
|id |sentence                            |words                                       |filtered                      |
+---+------------------------------------+--------------------------------------------+------------------------------+
|0  |Spark is great                      |[spark, is, great]                          |[spark, great]                |
|1  |We are learning Spark               |[we, are, learning, spark]                  |[learning, spark]             |
|2  |Spark is better than hadoop no doubt|[spark, is, better, than, hadoop, no, doubt]|[spark, better, hadoop, doubt]|
+---+------------------------------------+--------------------------------------------+------------------------------+



In [16]:
# Create a function to return the length of a list
def word_list_length(word_list):
    return len(word_list)

In [11]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [12]:
# Create a user defined function
count_tokens = udf(word_list_length, IntegerType())

In [25]:
# Select needed columns and don't truncate results
tokenized_df.withColumn("tokens", count_tokens(col("words"))).show(truncate=False)

+---+------------------------------------+--------------------------------------------+------+
|id |sentence                            |words                                       |tokens|
+---+------------------------------------+--------------------------------------------+------+
|0  |Spark is great                      |[spark, is, great]                          |3     |
|1  |We are learning Spark               |[we, are, learning, spark]                  |4     |
|2  |Spark is better than hadoop no doubt|[spark, is, better, than, hadoop, no, doubt]|7     |
+---+------------------------------------+--------------------------------------------+------+



In [31]:
# Import stop words library
from pyspark.ml.feature import StopWordsRemover
 # Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("StopWords").getOrCreate()
# Run the Remover
remover = StopWordsRemover(inputCol="words", outputCol="filtered")

In [26]:
tokenized_df.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|      Spark is great|  [spark, is, great]|
|  1|We are learning S...|[we, are, learnin...|
|  2|Spark is better t...|[spark, is, bette...|
+---+--------------------+--------------------+

